In [0]:
%fs ls /mnt/olist

In [0]:
pip install kaggle

In [0]:
import os

# Caminho atualizado correto
os.makedirs("/root/.config/kaggle", exist_ok=True)

# Conteúdo do seu kaggle.json
kaggle_json_content = """
{
  "username": "luciojr",
  "key": "ad8b74c5e48e49cdd3a3c917e8c5702d"
}
"""

# Grava o arquivo no local esperado pela API
with open("/root/.config/kaggle/kaggle.json", "w") as f:
    f.write(kaggle_json_content)

# Define permissões corretas
os.chmod("/root/.config/kaggle/kaggle.json", 0o600)



In [0]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# Define o local no DBFS onde salvar
local_path = "/tmp/olist"

os.makedirs(local_path, exist_ok=True)

# Baixa e extrai os arquivos no caminho
api.dataset_download_files("olistbr/brazilian-ecommerce", path=local_path, unzip=True)

import shutil

local_path = "/tmp/olist"
target_path = "/dbfs/mnt/olist/Bronze"

os.makedirs(target_path, exist_ok=True)

for file in os.listdir(local_path):
    shutil.copy(os.path.join(local_path, file), os.path.join(target_path, file))

In [0]:
import os

# Caminho onde estão os CSVs e onde os Parquets serão salvos
bronze_path = "dbfs:/mnt/olist/Bronze"

# Lista os arquivos na pasta Bronze
files = dbutils.fs.ls(bronze_path)

for file in files:
    file_name = file.name
    if file_name.endswith(".csv"):
        base_name = file_name.replace(".csv", "")
        csv_path = os.path.join(bronze_path, file_name)
        parquet_path = os.path.join(bronze_path, base_name)  

        # Ler CSV
        df = spark.read.csv(csv_path, header=True, inferSchema=True)

        # Salvar como Parquet (sobrescreve se já existir)
        df.write.mode("overwrite").parquet(parquet_path)

        # Remover o CSV original
        dbutils.fs.rm(csv_path, recurse=True)